<a href="https://colab.research.google.com/github/Abdullah12389/Multimart/blob/main/review_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.13 onnxruntime tf2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.2/524.2 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extens

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.13.0


In [ ]:
reviews=pd.read_csv("test.csv",header=None,names=['sentiment','review'],nrows=30000)

In [ ]:
reviews['sentiment']=np.where(reviews['sentiment']=="__label__1",0,1)

In [ ]:
#data is balanced
reviews['sentiment'].value_counts()

,count
sentiment,
1,15333
0,14667


In [ ]:
reviews=reviews.dropna().reset_index(drop=True)

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download("punkt_tab")
nltk.download("wordnet")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import re
import string
def normalize_text(text):
   text=text.lower()
   text=re.sub(f"[{re.escape(string.punctuation)}]",'',text)
   lemmatizer=WordNetLemmatizer()
   review=" ".join([lemmatizer.lemmatize(word) for word in text.split() if word.isalpha()])
   return review

In [ ]:
reviews['review']=reviews['review'].apply(lambda x:normalize_text(x))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer=Tokenizer(num_words=10000)
tokenizer.fit_on_texts(reviews['review'])
reviews['review']=tokenizer.texts_to_sequences(reviews['review'])

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(reviews['review'],reviews['sentiment'],test_size=0.2,random_state=42)

In [ ]:
Xtrain=pad_sequences(Xtrain,maxlen=100)
Xtest=pad_sequences(Xtest,maxlen=100)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM,Input,Dense,Dropout,Embedding
from tensorflow.keras.models import Sequential

In [ ]:
Xtrain=Xtrain.reshape(24000,1,100)
Xtest=Xtest.reshape(6000,1,100)

In [ ]:
input=Input(shape=(100,))
x=Embedding(10000,10)(input)
x=LSTM(100,return_sequences=True)(x)
x=Dropout(0.2)(x)
x=Dense(64,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(32,activation='relu')(x)
output=Dense(1,activation='sigmoid')(x)
model=tf.keras.Model(inputs=input,outputs=output)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(Xtrain,Ytrain,epochs=1,batch_size=64,validation_data=(Xtest,Ytest))

375/375 [==============================] - 51s 136ms/step - loss: 0.4590 - accuracy: 0.7502 - val_loss: 0.5387 - val_accuracy: 0.7106


In [ ]:
model.save("sentiment_analysis.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
input_sign=[tf.TensorSpec([None,100],tf.int32,name="input")]
import tf2onnx

In [ ]:
onnx_model,_=tf2onnx.convert.from_keras(model,input_sign,opset=13,output_path="sentiment_analysis.onnx")

In [ ]:
with open("sentiment_tokens.json","w") as f:
  f.write(str(sentiment_tokens))